In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/21 12:20:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Compiling /home/eczech/repos/ot-scoring/notebooks/pathinit.scCompiling /home/eczech/repos/ot-scoring/notebooks/cpinit.sc

import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

## Load RS Data

In [2]:
var df = ss.read.parquet(RESULTS_DIR.resolve("score_association.parquet").toString)

parquet at cmd1.sc:1

1 / 1

df: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 3 more fields]

In [3]:
df.show(3)

show at cmd2.sc:1

1 / 1

+---------------+--------------+-------------------+---------+--------------------+
|      target_id|    disease_id|              score|is_direct|          source_ids|
+---------------+--------------+-------------------+---------+--------------------+
|ENSG00000000005|  OTAR_0000018| 0.2723461111111111|    false|[phenodigm, europ...|
|ENSG00000000419|Orphanet_97242|             0.0328|     true|         [europepmc]|
|ENSG00000000938|   EFO_0000677|0.20965928796689323|    false|[phenodigm, europ...|
+---------------+--------------+-------------------+---------+--------------------+
only showing top 3 rows



In [4]:
df.count()

count at cmd3.sc:1

16 / 16

count at cmd3.sc:1

1 / 1

res3: Long = 6350031L

In [5]:
val df_rs_as = df.select($"target_id", $"disease_id", $"score".as("score_rs"))
df_rs_as.show(3)

show at cmd4.sc:2

1 / 1

+---------------+--------------+-------------------+
|      target_id|    disease_id|           score_rs|
+---------------+--------------+-------------------+
|ENSG00000000005|  OTAR_0000018| 0.2723461111111111|
|ENSG00000000419|Orphanet_97242|             0.0328|
|ENSG00000000938|   EFO_0000677|0.20965928796689323|
+---------------+--------------+-------------------+
only showing top 3 rows



df_rs_as: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 1 more field]

In [6]:
var df = ss.read.parquet(RESULTS_DIR.resolve("score_source.parquet").toString)

parquet at cmd5.sc:1

1 / 1

df: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 4 more fields]

In [7]:
df.show(3)

show at cmd6.sc:1

1 / 1

+---------------+---------------+--------------+--------------------+---------+--------------------+
|      target_id|     disease_id|     source_id|           score_raw|is_direct|               score|
+---------------+---------------+--------------+--------------------+---------+--------------------+
|ENSG00000000003|    EFO_0000677|     europepmc|0.014400000000000003|    false|0.014400000000000003|
|ENSG00000000419|Orphanet_183763|gene2phenotype|                 1.0|    false|                 1.0|
|ENSG00000000457|   DOID_0050890|     phenodigm|             0.22835|    false|             0.22835|
+---------------+---------------+--------------+--------------------+---------+--------------------+
only showing top 3 rows



In [8]:
val df_rs_sc = df.select($"target_id", $"disease_id", $"source_id", $"score".as("score_rs"))
df_rs_sc.show(3)

show at cmd7.sc:2

1 / 1

+---------------+---------------+--------------+--------------------+
|      target_id|     disease_id|     source_id|            score_rs|
+---------------+---------------+--------------+--------------------+
|ENSG00000000003|    EFO_0000677|     europepmc|0.014400000000000003|
|ENSG00000000419|Orphanet_183763|gene2phenotype|                 1.0|
|ENSG00000000457|   DOID_0050890|     phenodigm|             0.22835|
+---------------+---------------+--------------+--------------------+
only showing top 3 rows



df_rs_sc: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 2 more fields]

In [9]:
df_rs_sc.filter($"score_rs" <= 0).count()

count at cmd8.sc:1

16 / 16

count at cmd8.sc:1

1 / 1

res8: Long = 0L

In [10]:
df_rs_sc.count()

count at cmd9.sc:1

16 / 16

count at cmd9.sc:1

1 / 1

res9: Long = 8035532L

### Load OT Data

In [11]:
val associationPath = DATA_CACHE_DIR.resolve("association.parquet")

associationPath: java.nio.file.Path = /home/eczech/data/ot/cache/association.parquet

In [12]:
// Materialize this to cache the first time since the association data is huge
if (!associationPath.toFile.exists()){
    println("Extracting association data to cached location")
    ss.read.json(EXTRACT_DIR.resolve("association.json").toString).select(
        $"target.id".as("target_id"),
        $"disease.id".as("disease_id"),
        $"harmonic-sum".as("harmonic-sum")
    ).write.format("parquet").save(associationPath.toString)
}

json at cmd11.sc:2

876 / 876

save at cmd11.sc:6

876 / 876

In [13]:
var df_ot_ev = ss.read.parquet(associationPath.toString)

parquet at cmd12.sc:1

1 / 1

df_ot_ev: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 1 more field]

In [14]:
df_ot_ev.printSchema

root
 |-- target_id: string (nullable = true)
 |-- disease_id: string (nullable = true)
 |-- harmonic-sum: struct (nullable = true)
 |    |-- datasources: struct (nullable = true)
 |    |    |-- cancer_gene_census: double (nullable = true)
 |    |    |-- chembl: double (nullable = true)
 |    |    |-- crispr: double (nullable = true)
 |    |    |-- europepmc: double (nullable = true)
 |    |    |-- eva: double (nullable = true)
 |    |    |-- eva_somatic: double (nullable = true)
 |    |    |-- expression_atlas: double (nullable = true)
 |    |    |-- gene2phenotype: double (nullable = true)
 |    |    |-- genomics_england: double (nullable = true)
 |    |    |-- gwas_catalog: double (nullable = true)
 |    |    |-- intogen: double (nullable = true)
 |    |    |-- phenodigm: double (nullable = true)
 |    |    |-- phewas_catalog: double (nullable = true)
 |    |    |-- postgap: double (nullable = true)
 |    |    |-- progeny: double (nullable = true)
 |    |    |-- reactome: double (nu

In [15]:
df_ot_ev.show(3)

show at cmd14.sc:1

1 / 1

+---------------+-----------+--------------------+
|      target_id| disease_id|        harmonic-sum|
+---------------+-----------+--------------------+
|ENSG00000163755|EFO_0003100|[[0.0, 0.0, 0.0, ...|
|ENSG00000161395|EFO_0000684|[[0.0, 0.0, 0.0, ...|
|ENSG00000068001|EFO_0003769|[[0.0, 0.0, 0.0, ...|
+---------------+-----------+--------------------+
only showing top 3 rows



In [16]:
df_ot_ev.count()

count at cmd15.sc:1

29 / 29

count at cmd15.sc:1

1 / 1

res15: Long = 3359371L

In [17]:
val df_ot_as = df_ot_ev.select(
    $"target_id",
    $"disease_id",
    $"harmonic-sum.overall".as("score_ot")
)
df_ot_as.show(3)

show at cmd16.sc:6

1 / 1

+---------------+-----------+-------------------+
|      target_id| disease_id|           score_ot|
+---------------+-----------+-------------------+
|ENSG00000163755|EFO_0003100|  0.302613281377551|
|ENSG00000161395|EFO_0000684| 0.5359799999999999|
|ENSG00000068001|EFO_0003769|0.27293036574208873|
+---------------+-----------+-------------------+
only showing top 3 rows



df_ot_as: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 1 more field]

In [18]:
df_ot_as.count()

count at cmd17.sc:1

29 / 29

count at cmd17.sc:1

1 / 1

res17: Long = 3359371L

In [19]:
import org.apache.spark.sql.Row
val to_map = udf((r: Row) => {
    r.getValuesMap[Double](r.schema.fieldNames)
})
val df_ot_sc = df_ot_ev
    .select(
        $"target_id",
        $"disease_id",
        explode(to_map($"harmonic-sum.datasources"))
    )
    .withColumnRenamed("key", "source_id")
    .withColumnRenamed("value", "score_ot")
    // Exploding the struct of non-nullable doubles
    // results in a bunch of zeros that can be ignored
    .filter($"score_ot" > 0)
    .cache()
df_ot_sc.show(3)

show at cmd18.sc:17

1 / 1

+---------------+-----------+---------+-------------------+
|      target_id| disease_id|source_id|           score_ot|
+---------------+-----------+---------+-------------------+
|ENSG00000163755|EFO_0003100|phenodigm|  0.302613281377551|
|ENSG00000161395|EFO_0000684|      eva|0.48999999999999994|
|ENSG00000161395|EFO_0000684|phenodigm|            0.18392|
+---------------+-----------+---------+-------------------+
only showing top 3 rows



import org.apache.spark.sql.Row

to_map: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd18$Helper$$Lambda$5693/1334033233@49b17c77,
  MapType(StringType, DoubleType, false),
  None
)
df_ot_sc: org.apache.spark.sql.Dataset[Row] = [target_id: string, disease_id: string ... 2 more fields]

In [20]:
df_ot_sc.count()

count at cmd19.sc:1

29 / 29

count at cmd19.sc:1

1 / 1

res19: Long = 4044374L

In [21]:
df_ot_sc.count()

count at cmd20.sc:1

29 / 29

count at cmd20.sc:1

1 / 1

res20: Long = 4044374L

## Check Source Scores

In [22]:
var df = df_ot_sc.join(df_rs_sc, Seq("target_id", "disease_id", "source_id"), "outer")
df.show(3)

show at cmd21.sc:4

29 / 29

show at cmd21.sc:4

16 / 16

show at cmd21.sc:4

1 / 1

+---------------+-----------+----------------+--------------------+--------------------+
|      target_id| disease_id|       source_id|            score_ot|            score_rs|
+---------------+-----------+----------------+--------------------+--------------------+
|ENSG00000000003|EFO_0000191|expression_atlas|0.004551747879558587|0.004551747879558584|
|ENSG00000000003|EFO_0000199|expression_atlas|             0.03104|             0.03104|
|ENSG00000000003|EFO_0000228|       europepmc|                null|               0.006|
+---------------+-----------+----------------+--------------------+--------------------+
only showing top 3 rows



df: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 3 more fields]

In [23]:
df.printSchema()

root
 |-- target_id: string (nullable = true)
 |-- disease_id: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- score_ot: double (nullable = true)
 |-- score_rs: double (nullable = true)



In [24]:
df.count()

count at cmd23.sc:1

16 / 16

count at cmd23.sc:1

29 / 29

count at cmd23.sc:1

1 / 1

res23: Long = 8048209L

In [25]:
var df_na = df
    .withColumn("rs_null", $"score_rs".isNull)
    .withColumn("ot_null", $"score_ot".isNull)

df_na: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 5 more fields]

In [26]:
df_na.groupBy("rs_null", "ot_null").count().show()

show at cmd25.sc:1

16 / 16

show at cmd25.sc:1

29 / 29

show at cmd25.sc:1

1 / 1

show at cmd25.sc:1

1 / 1

+-------+-------+-------+
|rs_null|ot_null|  count|
+-------+-------+-------+
|  false|  false|4031697|
|  false|   true|4003835|
|   true|  false|  12677|
+-------+-------+-------+



In [27]:
df_na.groupBy("source_id", "rs_null", "ot_null").count().show()

show at cmd26.sc:1

29 / 29

show at cmd26.sc:1

16 / 16

show at cmd26.sc:1

1 / 1

show at cmd26.sc:1

1 / 1

+------------------+-------+-------+-------+
|         source_id|rs_null|ot_null|  count|
+------------------+-------+-------+-------+
|  expression_atlas|  false|  false| 141340|
|         europepmc|  false|   true|3990238|
|         phenodigm|  false|  false|1868289|
|  genomics_england|  false|  false|  90033|
|           uniprot|  false|  false|  82745|
|uniprot_literature|  false|  false|  80619|
|               eva|  false|  false| 135857|
|    gene2phenotype|  false|  false|  42700|
|          reactome|  false|  false|  17197|
|      gwas_catalog|  false|  false| 446832|
|            chembl|  false|  false| 226408|
|        slapenrich|  false|  false| 312960|
|            sysbio|  false|  false|   1935|
|    phewas_catalog|  false|  false| 235449|
|    phewas_catalog|  false|   true|  12184|
|    phewas_catalog|   true|  false|  12637|
|            crispr|  false|  false|  23694|
|       eva_somatic|  false|  false|  12487|
|cancer_gene_census|  false|  false| 279456|
|         

In [28]:
var df_diff = df
    .withColumn("rs_null", $"score_rs".isNull)
    .withColumn("ot_null", $"score_ot".isNull)
    .filter(!$"rs_null" && !$"ot_null")
    .withColumn("diff", abs($"score_rs" - $"score_ot"))
    .withColumn("isequal", $"diff" < .000001)

df_diff: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 7 more fields]

In [29]:
df_diff.groupBy($"isequal").count().show()

show at cmd28.sc:1

29 / 29

show at cmd28.sc:1

16 / 16

show at cmd28.sc:1

1 / 1

show at cmd28.sc:1

1 / 1

+-------+-------+
|isequal|  count|
+-------+-------+
|   true|3981238|
|  false|  50459|
+-------+-------+



In [30]:
df_diff.groupBy($"source_id", $"isequal").count().orderBy("source_id").show()

show at cmd29.sc:1

29 / 29

show at cmd29.sc:1

16 / 16

show at cmd29.sc:1

1 / 1

show at cmd29.sc:1

1 / 1

+------------------+-------+-------+
|         source_id|isequal|  count|
+------------------+-------+-------+
|cancer_gene_census|   true| 279456|
|            chembl|   true| 226408|
|            crispr|   true|  23694|
|               eva|   true| 135857|
|       eva_somatic|   true|  12487|
|  expression_atlas|  false|     69|
|  expression_atlas|   true| 141271|
|    gene2phenotype|   true|  42700|
|  genomics_england|   true|  90033|
|      gwas_catalog|   true| 426062|
|      gwas_catalog|  false|  20770|
|           intogen|   true|  30025|
|           intogen|  false|     22|
|         phenodigm|   true|1868289|
|    phewas_catalog|   true| 205853|
|    phewas_catalog|  false|  29596|
|           progeny|   true|   2856|
|          reactome|   true|  17197|
|        slapenrich|   true| 312960|
|            sysbio|   true|   1933|
+------------------+-------+-------+
only showing top 20 rows



## Check Association Scores

In [31]:
var df = df_ot_as.join(df_rs_as, Seq("target_id", "disease_id"), "inner")
df.show(3)

show at cmd30.sc:4

16 / 16

show at cmd30.sc:4

29 / 29

show at cmd30.sc:4

1 / 1

+---------------+-----------+--------------------+--------------------+
|      target_id| disease_id|            score_ot|            score_rs|
+---------------+-----------+--------------------+--------------------+
|ENSG00000000003|EFO_0000191|0.004551747879558587|0.004551747879558584|
|ENSG00000000003|EFO_0000199|             0.03104|             0.03104|
|ENSG00000000003|EFO_0000249|0.011536990372872362|0.011536990372872364|
+---------------+-----------+--------------------+--------------------+
only showing top 3 rows



df: org.apache.spark.sql.package.DataFrame = [target_id: string, disease_id: string ... 2 more fields]

In [32]:
df.withColumn("isequal", abs($"score_rs" - $"score_ot") < .000001).groupBy("isequal").count().show()

show at cmd31.sc:1

16 / 16

show at cmd31.sc:1

29 / 29

show at cmd31.sc:1

1 / 1

show at cmd31.sc:1

1 / 1

+-------+-------+
|isequal|  count|
+-------+-------+
|   true|2325608|
|  false|1024841|
+-------+-------+

